In [ ]:
using Plots
using LaTeXStrings
using Calculus       ### Módulo en Julia que permite calcular derivadas, gradientes y jacobinas.

In [ ]:
"""
i-esimo polinomios de Lagrange
i perteneciente en {0,1,2,...,n}
x es argumento de Li(x)
Vx es un vector de componentes xi
"""
function Li(i,x,Vx)#Float64
    L=1.
    for j in 1:i-1
        L*=(x-Vx[j])/(Vx[j]-Vx[i])      ###  L=(x-Vx[j])/(Vx[j]-Vx[i])*L
    end
    for j in i+1:length(Vx)
        L*=(x-Vx[j])/(Vx[j]-Vx[i])
    end
    return L
end

In [ ]:
"""
Polinomio interpolante de Lagrange
x es argumento de P(x)
V es un vector de componentes xi
W es un vector de componentes yi
"""
function P(x,V,W)
    @assert length(V) == length(W)
    P=0.
    for i in 1:length(V)
        P += W[i]*Li(i,x,V)
    end
    return P
end

In [ ]:
Pf(x) = P(x,V,f.(V))    ### f.(V) f aplicado en todas las componentes de V

In [ ]:
N=200
xrange=x0:(x2-x0)/(N-1):x2

In [ ]:
plot(xrange,Pf,title="Polinomio interpolante de Lagrange",label="Pf")
plot!(f,label="f(x)",linestyle=:dash)
scatter!(V,f.(V),label="Puntos",legend=:topleft)

In [ ]:
function Polinomio_Newton(x,V,W)
    @assert length(V) == length(W)
    n = length(V)
    Fxx = zeros(n)
    a = zeros(n)
    a[1] = W[1]
    P = a[1]
    Q = 1.
    U = [W[i] for i in 1:n]
    for j in 1:n-1
        for i in 1:n-j
            Fxx[i] = (U[i+1]-U[i])/(V[i+j]-V[i])     ### V = [x0,x1,x2,...,xn] , W = [y0,y1,y2,...,yn]
            U[i] = Fxx[i]                              ### Uj[i] = [y_i - y_i-1]/(x_i - x_i-)                          
        end
        a[j+1] = U[1]
    end
    for k in 1:n-1
        Q *= (x-V[k])
        P += a[k+1]*Q
    end
    return P
end

In [ ]:
"Formula de diferencias finitas centrada de tre puntos"
function derivada_centrada(f,x,h)
    df=(f(x+h)-f(x-h))/(2h)
    return df
end

In [ ]:
e=eps(2.)   ### Epsilon de la maquina, es la distancia entre valores de punto flotante representables consecutivos en x

In [ ]:
derivative(f,x0)   ### ***Calculado en Julia usando las funciones matemáticas incorporadas

In [ ]:
diferencia(h)=abs(derivative(f,x0)-derivada_centrada(f,x0,h))
h=[1/10^(k) for k=0:19]

In [ ]:
scatter(h,diferencia,linestyle=:dash, yscale=:log10,xscale=:log10, xlim=(1e-17,1e-0),title="Error absoluto")
vline!([1e-5],label="k=5")

In [ ]:
"Formula de diferencias finitas hacia adelante"
function derivada_adelante(f,x,h)
    df=(f(x+h)-f(x))/h
    return df
end

In [ ]:
"Formula de diferencias finitas centrada de tres puntos"
function derivada_centrada_3p(f,x,h)
    df=(f(x+h)-f(x-h))/(2h)
    return df
end

In [ ]:
"Formula de diferencias finitas centrada de cinco puntos"
function derivada_centrada_5p(f,x,h)
    df=(f(x-2h)-8f(x-h)+8f(x+h)-f(x+2h))/(12h)       ### df=(f(x-2h)-8f(x-h)+8f(x+h)-f(x-2h))/(16h) esta mal pero da igual :p
    return df
end

In [ ]:
**Ayuda:** Para bajar el archivo `pos.dat` del repositorio de github desde julia realice:

In [ ]:
# Ejemplo de como bajar un archivo.
separador = "/" # En Linux
#separador = "\" # En Windows
download(
    "https://raw.githubusercontent.com/reula/Metodos_Numericos_2022/main/Guias/pos.dat", # Bajamos el archivo pos.dat del repositorio en el que están las guías.
    pwd() * separador * "pos.dat" # Guardamos lo bajado en un archivo llamado pos.dat en el directorio local.
)

In [ ]:
**Ayuda:** Para cargar los datos en `pos.dat` a vectores de Julia utilice:

In [ ]:
t = Vector{Float64}()
x = Vector{Float64}()
open("pos.dat","r") do fh
    for line in readlines(fh) 
        cols = split(line)
        push!(t,parse(Float64,cols[1]))
        push!(x,parse(Float64,cols[2]))
    end
end

In [ ]:
function punto_medio(f,a,b,n)
    Sm=0
    h=(b-a)/(n-1)                       ### n puntos y n-1 intervalos
    x0=a
    for i=1:n-1                       
        x1=a+i*h
        Sm+=(x1-x0)f((x0+x1)/2)        ### Sm+=h*f(x0+h/2)
        x0=x1
        #println("$i, $Sm, $x1")
    end
    return Sm
end

In [ ]:
function trapecio(f,a,b,n)
    St=0
    h=(b-a)/(n-1)
    x0=a
    for i=1:n-1
        x1=a+i*h
        St+=(x1-x0)*(f(x0)+f(x1))/2
        x0=x1
        #println("$i, $St, $x1")
    end
    return St
end

In [ ]:
function Simpson(f,a,b,n)       ### Con n impar i.e intervalos pares 
    @assert n%2==1
    Ss=0
    h=(b-a)/(n-1)
    x0=a
    for i=1:n-1
        x2=a+i*h
        x1=(x0+x2)/2
        Ss+=h/6*(f(x0)+4f(x1)+f(x2))
        x0=x2
        #println("$i, $Ss, $x2")
    end
    return Ss
end

In [ ]:
"1er paso metodo de Euler"
function Euler(f,t0,y0,h,p)
    return y0 + h*f(t0, y0, p)                ### p: parametros o constantes de f
end

In [ ]:
"1er paso metodo Runge-Kutta 2"
function RK2(f,t0,y0,h,p)
    k1 = f(t0, y0, p)
    k2 = f(t0+h, y0+h*k1, p)
    return y0 + (h/2)*(k1 + k2)               ### y0 + h/2*(k1 + k2) No anda
end

In [ ]:
"1er paso metodo Runge-Kutta 4"
function RK4(f,t0,y0,h,p)
    k1 = f(t0, y0, p)
    k2 = f(t0+h/2, y0+h/2*k1, p)
    k3 = f(t0+h/2, y0+h/2*k2, p)
    k4 = f(t0+h, y0+h*k3, p)
    return y0 + (h/6)*(k1 + 2*k2 + 2*k3 + k4)      ### y0 + h/6(k1 + 2*k2 + 2*k3 + k4) No anda
end

In [1]:
function iteracion_ODES(Metodo,f,y0,(a,b),N,p)
    t = zeros(N)
    w = zeros(N)
    h = (b-a)/(N-1)
    t[1] = a
    w[1] = y0
    for i in 2:N
        t[i] = t[i-1]+h
        w[i] = Metodo(f,t[i-1],w[i-1],h,p)
    end
    return t[:],w[:]
end

iteracion_ODES (generic function with 1 method)

In [ ]:
f(t,y,p) = -y+sin(2pi*t)
a = 0
b = 1
y0 = 1.0
N = 10          ### h=0,1
p = 0

In [ ]:
t1,w1=iteracion_ODES(Euler,f,y0,(a,b),N,p)

In [ ]:
y1 = [y(t1[n]) for n=1:N]
plot(title="Error Global",xlabel="t",ylabel="e(t)")
plot!(t1,abs.(w1-y1),label="Euler",linestyle=:dash)
plot!(t2,abs.(w2-y1),label="RK2",linestyle=:dash)
plot!(t3,abs.(w3-y1),label="RK4",linestyle=:dash)

In [ ]:
function iteracion_ODES_multidumencional(Metodo,f,y0,(a,b),N,p)
    t = zeros(N)
    w = zeros(length(y0),N)
    h = (b-a)/(N-1)
    t[1] = a
    w[:,1] = y0
    for i in 2:N
        t[i] = t[i-1]+h
        w[:,i] = Metodo(f,t[i-1],w[:,i-1],h,p)
    end
    return t[:],w[:,:]
end

In [ ]:
function f(t,w,p)
    (g,l)=p
return [w[2];(-g/l)*sin(w[1])]
end

In [ ]:
y0=[0.25,0]
a=0
b=10
N=1000                                 ### h=0,01
p=(10,1)
t,w=iteracion_ODES_multidumencional(RK4,f,y0,(a,b),N,p)

In [ ]:
plot(t,w[1,:])
plot!(t,w[2,:])